# Boeing Defence Australia Data Challenge

As part of key selection activities for Boeing Defence Australia, Boeing has requested the completion of a number of data challenges. 

There are three key questions submitted by Boeing for response, but the first task is to obtain the data.

In [10]:
import pandas as pd
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

## Obtain and Clean Data from API

Boeing provided details of a data.gov.au REST api to be targetted for data acquisition. 

I explored the API documentation and found a way to acquire the data by targetting a specific URL.

I prefer to take a functional approach to writing programs. So I have created two simple functions below -

1. Acquire the Data.
2. Clean the dataframe.


In [161]:
#select all from the given API resource - SQL api works as well as any other. 
url = 'https://data.gov.au/data/api/3/action/datastore_search_sql?sql=SELECT * from "809c77d8-fd68-4a2c-806f-c63d64e69842"'


def load_data_from_api(url: str) -> pd.DataFrame:
    
    """
    Function which takes a url as an input argument and returns a pandas dataframe.
    
    Uses the requests library to request the data. Utilised the SQL part of the REST api to gather all elements from the relevant dataset. 
    
    Extracts result part and then the records part of the resulting dictionary and returns as pandas dataframe.

    """
    
    #get json response dictionary
    json_api_response = requests.get(url).json()
    
    #get records set
    records = json_api_response['result']['records']
    
    #load_records_as df
    df = pd.DataFrame(records)
    
    return df

    
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    
    
    """
    Function performs all required cleaning for the data frame prior to processing for solution to questions.
    
    Renames columns systematically to get rid of brackets and lower case the columns because I simply prefer this for consistency.
    
    Finds numerical types and transforms them to appropriate numerical types as the pandas dataframe reads them in as strings by default. 
    
    Creates a dummy date variable based on the year and the month number for easier interrogation of dates later on.
    
    """
    
    #lower case column names remove brackets
    df.columns = [col.lower().replace('(', '').replace(')', '') for col in df.columns]
    
    #change some datatypes - specify what you want as float in float and what you want as int in int
    numerical_columns_dict = {'float': [col for col in df.columns if 'tonnes' in col], 
                              'int': ['month_num', 'passengers_out','passengers_in']}
    
    
    for dtype, type_columns in numerical_columns_dict.items():
        
        for column in type_columns:
            #Change each of the columns in tern to
            df[column] = df[column].fillna(0).astype(dtype)
    
    #drop _full_text because I don't care and assign a year_month date_variable at the start point of each month
    
    df = (df
          .drop(columns = ['_full_text'])
          .assign(year_month = lambda df: (pd.to_datetime(df['year'].astype(str)+'-'+df['month_num'].astype(str)+'-01'))))
    
    return df
   

    

## Load the data and clean using functions

In [162]:
df = (load_data_from_api(url)
      .pipe(clean_dataframe))

# Questions

Questions will be answered in standard pandas syntax using method chaining.

I have provided the output of these files in the github along with this notebook file.

## Question 1

> 1. How many passengers flew into and out of Australia for each airline in each month of 2019?



In [212]:
(df
 .query('year_month.dt.year == 2019', engine='python') #what year
 [['year_month', 'airline', 'passengers_in', 'passengers_out']] #get columns
 .groupby(['year_month','airline'], as_index = False).sum() #sum 
 .sort_values(['year_month','airline']) #sort for readability
 .to_csv('question1.csv',encoding = 'utf-8', index=False)) #write out file

## Question 2

> 2. In the last 6 months which port country was responsible for the most passengers coming into
the country?

For this question, I have elected to calculate 6 months back in the past and round back to the start of that month. 

Interrogating the data, I believe that data is provided on a one to two month lag. At the time of writing this file, the maximum date in the dataset is 2021-05-01. The parameter below that creates the `six_months_ago` variable could be altered to be seven months to be sure.

Also... New Zealand travel bubble... Who knew?

In [213]:

#create datetime objects represented the start of the month that was six months ago from the current date. 
six_months_ago = date.today() - 6*relativedelta(months = 1)
six_months_ago_start = datetime.strptime(f'{six_months_ago.year}-{six_months_ago.month}-01', '%Y-%m-%d')

(df
 .query('year_month>=@six_months_ago_start')#enforce six months ago only
 [['port_country','passengers_in']] #get columns we care about - we don't care about date we want the maximum over the window
 .groupby('port_country', as_index = False).sum() # sum
 .assign(max_passengers_in = lambda df: df['passengers_in'].max()) # assign a series which is just the maximum sum
 .query('max_passengers_in == passengers_in') #query to get the row where the passenger count is the maximum passenger count
 .drop(columns = ['max_passengers_in']) # we don't need two copies of the same number - drop one of them.
 .to_csv('question2.csv', encoding ='utf-8', index=False)) #write out



## Question 3

> 3. What is the 3 month rolling average of freight arriving to Australia for each airline throughout
2018?

So, I have assumed you want the three month rolling average for all months in 2018. I therefore started the data in November of 2017 to give three months for averaging by the start of 2018. 

In [218]:
#create datetime objects for querying
start_date = datetime.strptime(f'2017-11-01', '%Y-%m-%d')
end_date = datetime.strptime(f'2018-12-31', '%Y-%m-%d')

(df
 .query('@start_date<=year_month<=@end_date')#get date range
 [['year_month','airline','freight_in_tonnes']] #select columns I care about
 .groupby(['airline', 'year_month']).sum() #sum over airline and month
 .assign(rolling_average_freight_in_tonnes = lambda df: df['freight_in_tonnes'].rolling(3).mean()) # assign a rolling average variable
 .reset_index() # index is painful
 .query('year_month.dt.year==2018')#give only what you asked for - 2018
 .to_csv('question3.csv', index=False, encoding = 'utf-8')
)

Thanks for your consideration of my application.

-Mitchell Gray.